In [ ]:
import os, subprocess
import random
import string
from IPython.display import clear_output
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
def clear():
  if clear_out:
    clear_output()
# Задание имени папки
sdwu = "stable-"+"diffusion"+"-we"+"bui"
sdwu1 = "s"+"d-we"+"bui"
wup = "we"+"bu"+"i.py"
def random_string(length):
    letters = string.ascii_lowercase
    rand_string = ''.join(random.choice(letters) for i in range(length))
    return rand_string
rnd = random_string(12)
fpath = f"/content/{rnd}/"
# Переустановка торча
#!pip install numpy --pre torch torchvision torchaudio --force-reinstall --index-url https://download.pytorch.org/whl/nightly/cu118
# Фиксы колаба (утечка памяти)
#!sudo apt-get install libgoogle-perftools-dev
!apt -y update -qq > /dev/null 2>&1
!sudo apt-get install libjemalloc-dev > /dev/null 2>&1
!sudo rm /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2 > /dev/null 2>&1
!sudo rm /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2 > /dev/null 2>&1
!sudo rm /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libtbb.so.12 /usr/local/lib/libtbb.so.12 > /dev/null 2>&1
!sudo rm /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3 > /dev/null 2>&1
!sudo rm /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3 > /dev/null 2>&1
!sudo rm /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3 > /dev/null 2>&1
clear()
# Клонирование репозитория
subprocess.run(["git", "clone", f"https://github.com/AUTOMATIC1111/{sdwu}", fpath])
# Переключение на нужный рабочий коммит
subprocess.run(["git", "checkout", commit], cwd=fpath, capture_output=True, text=True)
# Установка репозиториев для вебуи
launch = f"{fpath}modules/launch_utils.py"
with open(f'{launch}', 'r') as fp:
  lines = fp.readlines()
  for line in lines:
    if line.find('STABLE_DIFFUSION_COMMIT_HASH') != -1:stable_diffusion_commit_hash = line.split('"')[1]
    if line.find('TAMING_TRANSFORMERS_COMMIT_HASH') != -1:taming_transformers_commit_hash = line.split('"')[1]
    if line.find('K_DIFFUSION_COMMIT_HASH') != -1:k_diffusion_commit_hash = line.split('"')[1]
    if line.find('CODEFORMER_COMMIT_HASH') != -1:codeformer_commit_hash = line.split('"')[1]
    if line.find('BLIP_COMMIT_HASH') != -1:blip_commit_hash = line.split('"')[1]
reps = [
    f"git clone https://github.com/Stability-AI/stablediffusion.git {fpath}repositories/stable-diffusion-stability-ai && cd {fpath}repositories/stable-diffusion-stability-ai && git checkout {stable_diffusion_commit_hash}",
    f"git clone https://github.com/crowsonkb/k-diffusion.git {fpath}repositories/k-diffusion && cd {fpath}repositories/k-diffusion && git checkout {k_diffusion_commit_hash}",
    f"git clone https://github.com/sczhou/CodeFormer.git {fpath}repositories/CodeFormer && cd {fpath}repositories/CodeFormer && git checkout {codeformer_commit_hash}",
    f"git clone https://github.com/salesforce/BLIP.git {fpath}repositories/BLIP && cd {fpath}repositories/BLIP && git checkout {blip_commit_hash}",
    f"git clone https://github.com/isl-org/MiDaS.git {fpath}repositories/midas",
    ]
def reps_inst(rep):
  !$rep
with ThreadPoolExecutor(max_workers=len(reps)) as executor:
  futures = [executor.submit(reps_inst, rep) for rep in reps]
  for future in as_completed(futures):
      future.result()
clear()
# Фиксы
os.environ["SAFETENSORS_FAST_GPU"]='1'
%env LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libjemalloc.so
%env PYTORCH_CUDA_ALLOC_CONF=garbage_collection_threshold:0.9,max_split_size_mb:512
#%env TF_CPP_MIN_LOG_LEVEL=3
!sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' {fpath}{wup}
if Версия_WebUI == "1.5.2":
  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" {fpath}modules/sd_models.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" {fpath}modules/extras.py
  !sed -i 's@cmd_opts.lowram else \"cpu\"@cmd_opts.lowram else \"cuda\"@' {fpath}modules/shared.py
  with open(f"{fpath}repositories/stable-diffusion-stability-ai/ldm/util.py") as r: text = r.read().replace('return get_obj_from_str(config["target"])(**config.get("params", dict()))', 'return get_obj_from_str(config["target"])(**config.get("params", dict())).cuda()')
  with open(f"{fpath}repositories/stable-diffusion-stability-ai/ldm/util.py", "w") as w: w.write(text)
# Дополнения
extentions = [
  #f"git clone https://github.com/KohakuBlueleaf/a1111-{sdwu1}-lycoris {fpath}extensions/a1111-{sdwu1}-lycoris",
  #f"git clone https://github.com/kohya-ss/{sdwu1}-additional-networks {fpath}extensions/{sdwu1}-additional-networks",
  f"git clone https://github.com/thomasasfk/{sdwu1}-aspect-ratio-helper {fpath}extensions/{sdwu1}-aspect-ratio-helper",
  f"git clone https://github.com/DominikDoom/a1111-{sdwu1}-tagcomplete {fpath}extensions/tag-autocomplete",
]
if CntN:
  extentions = extentions + [
      f"git clone https://github.com/Mikubill/{sdwu1}-controlnet {fpath}extensions/{sdwu1}-controlnet",
      #f"git clone https://github.com/jexom/{sdwu1}-depth-lib {fpath}extensions/{sdwu1}-depth-lib",
      #f"git clone https://github.com/fkunn1326/openpose-editor {fpath}extensions/openpose-editor",
      f"git clone https://github.com/ZhUyU1997/open-pose-editor {fpath}extensions/3D-openpose-editor"
  ]
def extentions_install(extention):
  !$extention
with ThreadPoolExecutor(max_workers=len(extentions)) as executor:
  futures = [executor.submit(extentions_install, extention) for extention in extentions]
  for future in as_completed(futures):
      future.result()
# Сохранение картинок на гулодиск + допнастройка
if gdrive:
  !cp /content/drive/MyDrive/SD/config.json {fpath}
  !cp /content/drive/MyDrive/SD/ui-config.json {fpath}
  with open(f"{fpath}config.json") as r: text = r.read().replace('/content/output/', '/content/drive/MyDrive/SD/')
  with open(f"{fpath}config.json", "w") as w: w.write(text)
  with open(f"{fpath}config.json") as r: text = r.read().replace('"control_net_allow_script_control": true', '"control_net_allow_script_control": true,\n    "def_batch_dir": "/content/drive/MyDrive/SD/ip2p-batch",\n    "outdir_ip2p_samples": "/content/drive/MyDrive/SD/ip2p-images"')
  with open(f"{fpath}config.json", "w") as w: w.write(text)
  if AOM3 and not (Hassaku or Koji):
    with open(f"{fpath}ui-config.json") as r: text = r.read().replace('"txt2img/Sampling steps/value": 25', '"txt2img/Sampling steps/value": 35')
    with open(f"{fpath}ui-config.json", "w") as w: w.write(text)
    with open(f"{fpath}ui-config.json") as r: text = r.read().replace('"txt2img/Sampling method/value": "DDIM"', '"txt2img/Sampling method/value": "DPM++ 2M Karras"')
    with open(f"{fpath}ui-config.json", "w") as w: w.write(text)
# Штатная установка зависимостей средствами launch.py
%cd {fpath}
!python launch.py --share --opt-sdp-attention --ngrok test --no-download-sd-model --deepdanbooru --exit
clear()
# Cкачивание моделей ControlNet
if CntN:
  dlcn = f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition -P {fpath}extensions/{sdwu1}-controlnet/models"
  cn_models = [
    f"{dlcn} https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11f1p_sd15_depth.pth",
    f"{dlcn} https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_openpose.pth",
    #f"{dlcn} https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_canny.pth",
    f"{dlcn} https://huggingface.co/lllyasviel/ControlNet-v1-1/resolve/main/control_v11p_sd15_normalbae.pth",
    #f"{dlcn} https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.yaml",
    #f"{dlcn} https://huggingface.co/monster-labs/control_v1p_sd15_qrcode_monster/resolve/main/control_v1p_sd15_qrcode_monster.safetensors"
  ]
  def cn_models_install(cn_model):
    !$cn_model
  with ThreadPoolExecutor(max_workers=len(cn_models)) as executor:
    futures = [executor.submit(cn_models_install, cn_model) for cn_model in cn_models]
    for future in as_completed(futures):
      future.result()
# Скачивание моделей
models_folder = f"{fpath}models/Stable-diffusion"
model_dl = f"wget -nv -t 10 --show-progress --progress=bar -q --content-disposition -P {models_folder}"
models = []
if AOM3:
  models = models + [
    #f"{model_dl} https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3_orangemixs.safetensors",
    f"{model_dl} https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1B_orangemixs.safetensors",
    f"{model_dl} https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors",
    f"{model_dl} https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors",
  ]
if Hassaku:
  models = models + [
      #f"{model_dl} https://huggingface.co/hanafuusen2001/Hassaku/resolve/main/hassakuHentaiModel_v11.safetensors",
      #f"{model_dl} https://huggingface.co/Romanos575/Hassaku/resolve/main/hassakuHentaiModel_v12.safetensors",
      f"{model_dl} https://huggingface.co/Romanos575/Hassaku/resolve/main/hassakuHentaiModel_v13.safetensors"
  ]
if Koji:
  models = models + [f"{model_dl} https://huggingface.co/Magamanny/Koji/resolve/main/koji_v21.safetensors"]
if Meina:
  models = models + [
      #f"{model_dl} https://civitai.com/api/download/models/64046", # MeinaHentai 3
      #f"{model_dl} https://civitai.com/api/download/models/26033", # MeinaHentai 2.1
      #f"{model_dl} https://civitai.com/api/download/models/67625", # MeinaAlter 2
      #f"{model_dl} https://civitai.com/api/download/models/22314", # MeinaUnreal Beta
      f"{model_dl} https://civitai.com/api/download/models/112809", # MeinaUnreal 4.1
  ]
if YesMix:
  models = models + [f"{model_dl} https://civitai.com/api/download/models/11086"] # YesMix 1.5
if Cardos:
  models = models + [
      f"{model_dl} https://huggingface.co/yz54321/cardosAnime/resolve/main/cardosAnime_v10.safetensors",
      #f"{model_dl} https://huggingface.co/ckpt/cardos-anime/resolve/main/cardosAnime_v20.safetensors",
  ]
if not models:
  models = models + [f"{model_dl} https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.safetensors",]
def download_models(model):
  !$model
with ThreadPoolExecutor(max_workers=len(models)) as executor:
  futures = [executor.submit(download_models, model) for model in models]
  for future in as_completed(futures):
      future.result()
# Подтягивание моделей, лор, VAE и эмбедов с гугл-диска
loras_folder = f"{fpath}models/Lora"
VAEs_folder = f"{fpath}models/VAE"
embeds_folder = f"{fpath}embeddings"
lycos_folder = f"{fpath}models/LyCORIS"
if gdrive:
  !mkdir -p {loras_folder}
  !ln -s /content/drive/MyDrive/SD/models/Stable-diffusion/* {models_folder}
  !ln -s /content/drive/MyDrive/SD/models/Lora/* {loras_folder}
  !ln -s /content/drive/MyDrive/SD/models/VAE/* {VAEs_folder}
  !ln -s /content/drive/MyDrive/SD/embeddings/* {embeds_folder}
  !ln -s /content/drive/MyDrive/SD/models/LyCORIS/* {loras_folder}
# Запуск вебуи
if not ngrok_token or len(ngrok_token) < 48 or len(ngrok_token) > 51:
  ngrok = ''
else:
  ngrok = f'--ngrok {ngrok_token} --ngrok-region eu'
!python $wup --opt-sdp-attention --no-half-vae --deepdanbooru --disable-safe-unpickle --enable-insecure-extension-access --share --listen --gradio-queue --disable-console-progressbars --no-hashing --theme dark $ngrok